In [1]:
# Imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from collections import defaultdict as ddict, Counter
from itertools import chain

import csv
import json
import itertools
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from operator import itemgetter

In [2]:
# Read papers

def read_papers(filename='authors.csv'):
    papers = ddict(set)
    authors = ddict(set)
    affiliations = ddict(set)
    author2affiliation = ddict(set)
    paper2authors = ddict(set)
    paper2authorsaffiliation = ddict(set)
    paper2affiliations = ddict(set)

    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
                continue
            paper_title = row[1].strip()
            paper_authors = row[2].strip()
            
            paper2authors[paper_title].add(paper_authors)
            
            full_name = paper_authors
            affiliation = row[3].strip()
            paper2authorsaffiliation[paper_title].add((paper_authors, affiliation))
            paper2affiliations[paper_title].add(affiliation)

            papers[paper_title.strip()].add(full_name)
            authors[full_name.strip()].add(paper_title)
            affiliations[affiliation.strip()].add(paper_title)
            author2affiliation[full_name.strip()].add(affiliation)
            line_count += 1
        return papers, authors, affiliations, author2affiliation, paper2authors, paper2authorsaffiliation, paper2affiliations

kfupm_papers, kfupm_authors, kfupm_affiliations, kfupm_author2affiliation, kfupm_paper2authors, kfupm_paper2authorsaffiliation, kfupm_paper2affiliations = read_papers(filename='kfupm_authors.csv')
ksu_papers, ksu_authors, ksu_affiliations, ksu_author2affiliation, ksu_paper2authors, ksu_paper2authorsaffiliation, ksu_paper2affiliations = read_papers(filename='ksu_authors.csv')
mit_papers, mit_authors, mit_affiliations, mit_author2affiliation, mit_paper2authors, mit_paper2authorsaffiliation, mit_paper2affiliations = read_papers(filename='mit_authors.csv')

# Affiliaiton -> Papers

def get_papers_for_name(affiliations):
    name2papers = ddict(set)
    for org in affiliations:
        name = org
        name2papers[name] = name2papers[name].union(set(affiliations[org]))
            
    return name2papers

kfupm_name2papers = get_papers_for_name(kfupm_affiliations)
ksu_name2papers = get_papers_for_name(ksu_affiliations)
mit_name2papers = get_papers_for_name(mit_affiliations)

In [3]:
# Grants

def read_grants(paper_filename, grants_filename, paper2affiliations):
    name2grants = ddict(set)
    with open(grants_filename) as grant_csv_file:
        grant_csv_reader = csv.reader(grant_csv_file, delimiter=',')
        grant_line_count = 0
        for row in grant_csv_reader:
            if grant_line_count == 0:
                grant_line_count += 1
                continue
            id = row[0].strip()
            grant = row[1].strip()
            with open(paper_filename) as paper_csv_file:
                paper_csv_reader = csv.reader(paper_csv_file, delimiter=',')
                line_count = 0
                for paper_row in paper_csv_reader:
                    if line_count == 0:
                        line_count += 1
                        continue
                    paper_id = paper_row[0].strip()
                    paper_title = paper_row[1].strip()
                    if paper_id == id:
                        for affiliation in paper2affiliations[paper_title]:
                            if name2grants[affiliation] == set():
                                name2grants[affiliation] = 0
                                name2grants[affiliation] += int(grant)
                            else:
                                name2grants[affiliation] += int(grant)
                        break;
                    line_count += 1
            grant_line_count += 1
        return name2grants

# kfupm_name2grants = read_grants('kfupm_authors.csv', 'kfupm_grants.csv', kfupm_paper2affiliations)
# print('Done')
# ksu_name2grants = read_grants('ksu_authors.csv', 'ksu_grants.csv', ksu_paper2affiliations)
# print('Done')
mit_name2grants = read_grants('mit_authors.csv', 'mit_grants.csv', mit_paper2affiliations)
print('Done')

Done


In [4]:
def write_json(data, filename='data.json'): 
    with open(filename,'w') as f: 
        json.dump(data, f, indent=4)
        
# write_json(kfupm_name2grants, 'kfupm_name2grants.json')
# write_json(ksu_name2grants, 'ksu_name2grants.json')
write_json(mit_name2grants, 'mit_name2grants.json')